In [1]:
# import the necessary packages
from imutils.video import VideoStream
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

In [2]:
import tensorflow as tf
import keras as ks
import cv2
import numpy as np
# print(ks.__version__)
tf.__version__ 

Using TensorFlow backend.


'2.1.0'

In [3]:
from tensorflow.keras.models import load_model

In [4]:
class_names = ['Mubarek', 'Other', '93333_08', '10Yafet', '117493_09', '117495_09', '117503_09', '117534_09', '117547_09', '117785_09','163441_06', '117624_09']
#, '122086_09', '117494_09', '117808_09', '117722_09', '121588_09''Nuredin', 
len(class_names)

12

In [5]:
face_recognition_model = load_model('P Face Recognition CNN/Models/Other_VGG_Face.h5')

In [6]:
model = load_model('liveness.model')

In [7]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
modelPath = os.path.sep.join(["face_detector", "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load the liveness detector model and label encoder from disk
print("[INFO] loading liveness detector...")
le = pickle.loads(open("le.pickle", "rb").read())

[INFO] loading face detector...
[INFO] loading liveness detector...


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0

#### Liveness detection then Face recognition

In [13]:
facecascade = cv2.CascadeClassifier('C:/Users/Nuredin Ali/haarcascade_frontalface_default.xml')

# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")

# vs = VideoStream(src=0).start()
# time.sleep(2.0)

# getting the reference to the Webcam
CAMERA = cv2.VideoCapture(0)
# camera_height = 500
width= 224
height= 224


# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 600 pixels
    _, frame = CAMERA.read()
    frame = imutils.resize(frame, width=600)
    # flip the frame
    frame = cv2.flip(frame, 1)
    
    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]
        
        # filter out weak detections
        if confidence > 0.5:
        
            # compute the (x, y)-coordinates of the bounding box for
            # the face and extract the face ROI
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            #ensure the detected bounding box does fall outside the
            # dimensions of the frame
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)

            # extract the face ROI and then preproces it in the exact
            # same manner as our training data
            fresh_face = frame[startY:endY, startX:endX]
            face = cv2.resize(fresh_face, (32, 32))
            face = face.astype("float") / 255.0
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)

            # pass the face ROI through the trained liveness detector
            # model to determine if the face is "real" or "fake"
            preds = model.predict(face)[0]
            j = np.argmax(preds)
            label = le.classes_[j]
            print(label)
            
            if label == 'real':
                # parse BRG to RGB
                roi = cv2.cvtColor(fresh_face, cv2.COLOR_BGR2RGB)

                # resize
                roi = cv2.resize(roi, (width, height))

                # KNOW PREDICT 
                roi_X = np.expand_dims(roi, axis=0)

                predictions = face_recognition_model.predict(roi_X)
                
                print(predictions[0])
                
                type_1_pred, type_2_pred, type_3_pred, type_4_pred, type_5_pred, type_6_pred, type_7_pred, type_8_pred, type_9_pred, type_10_pred, type_11_pred, type_12_pred = predictions[0]
                #, , type_7_pred
                # add text
                type_1_text = '{}: {}%' .format(class_names[0], int(type_1_pred*100))
                cv2.putText(frame, type_1_text, (5, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                # add text
                type_2_text = '{}: {}%' .format(class_names[1], int(type_2_pred*100))
                cv2.putText(frame, type_2_text, (5, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                # add text
                type_3_text = '{}: {}%' .format(class_names[2], int(type_3_pred*100))
                cv2.putText(frame, type_3_text, (5, 90),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                # add text
                type_4_text = '{}: {}%' .format(class_names[3], int(type_4_pred*100))
                cv2.putText(frame, type_4_text, (5, 110),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                # add text
                type_5_text = '{}: {}%' .format(class_names[4], int(type_5_pred*100))
                cv2.putText(frame, type_5_text, (5, 140),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                # add text
                type_6_text = '{}: {}%' .format(class_names[5], int(type_6_pred*100))
                cv2.putText(frame, type_6_text, (5, 170),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                #add text
                type_7_text = '{}: {}%' .format(class_names[6], int(type_7_pred*100))
                cv2.putText(frame, type_7_text, (5, 200),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                type_8_text = '{}: {}%' .format(class_names[7], int(type_8_pred*100))
                cv2.putText(frame, type_8_text, (5, 230),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                type_9_text = '{}: {}%' .format(class_names[8], int(type_9_pred*100))
                cv2.putText(frame, type_9_text, (5, 260),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                type_10_text = '{}: {}%' .format(class_names[9], int(type_10_pred*100))
                cv2.putText(frame, type_10_text, (5, 290),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                type_11_text = '{}: {}%' .format(class_names[10], int(type_11_pred*100))
                cv2.putText(frame, type_11_text, (5, 310),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

                type_12_text = '{}: {}%' .format(class_names[11], int(type_12_pred*100))
                cv2.putText(frame, type_12_text, (5, 340),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
                
                ##################################  Show whether the image is real or fake ##################################
                # draw the label and bounding box on the frame
                label = "{}: {:.4f}".format(label, preds[j])
                cv2.putText(frame, label, (startX, startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
                #####################################################################
                
                continue
            else:
                # draw the label and bounding box on the frame
                label = "{}: {:.4f}".format(label, preds[j])
                cv2.putText(frame, label, (startX, startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
                
                continue
              
            

    # show the output frame and wait for a key press
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
CAMERA.release()
cv2.destroyAllWindows()

[INFO] starting video stream...
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0.000000e+00 3.278642e-34 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00
 0.000000e+00 0.000000e+00]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
real
[1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 1.23366e-35 0.00000e+00 0.00000e+00]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[2.7985825e-09 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00]
r

[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 3.537373e-33
 0.000000e+00 1.000000e+00]
fake
real
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00360674 0.         0.99639326]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.378164e-13
 0.000000e+00 1.000000e+00]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
fake
real
[0. 0. 0. 0. 0. 0. 0

### Face recognition only

In [ ]:
facecascade = cv2.CascadeClassifier('C:/Users/Nuredin Ali/haarcascade_frontalface_default.xml')
 
import time
import cv2
# getting the reference to the Webcam
CAMERA = cv2.VideoCapture(0)

camera_height = 500
width= 224
height= 224


while(True):
    # read a new frame
    _, fr = CAMERA.read()

    # flip the frame
    frm = cv2.flip(fr, 1)
    
    gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
    
    faces = facecascade.detectMultiScale(gray)
    
    for (x, y, w, h) in faces:
        face_detect = cv2.rectangle(frm, (x, y), (x+w, y+h), (255, 0, 255), 2)
        roi_gray = frm[y:y+h, x:x+w]
        #print("XXXXX" , h)
        roi_color = frm[y:y+h, x:x+w]
    face_rectangle = faces
 
    if len(faces) != 0:
        x = face_rectangle[0][0]
        y = face_rectangle[0][1]
        hieght = face_rectangle[0][2]
        w = face_rectangle[0][3]
    else:
        continue
    
    # get ROI
    roi = frm[y:y+hieght, x:x+w]
    
    # parse BRG to RGB
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    # resize
    roi = cv2.resize(roi, (width, height))

    # KNOW PREDICT 
    roi_X = np.expand_dims(roi, axis=0)
    
    predictions = model.predict(roi_X)
    
    type_1_pred, type_2_pred, type_3_pred, type_4_pred, type_5_pred, type_6_pred, type_7_pred, type_8_pred, type_9_pred, type_10_pred, type_11_pred, type_12_pred = predictions[0]
    #, , type_7_pred
    # add text
    type_1_text = '{}: {}%' .format(class_names[0], int(type_1_pred*100))
    cv2.putText(frm, type_1_text, (5, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # add text
    type_2_text = '{}: {}%' .format(class_names[1], int(type_2_pred*100))
    cv2.putText(frm, type_2_text, (5, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # add text
    type_3_text = '{}: {}%' .format(class_names[2], int(type_3_pred*100))
    cv2.putText(frm, type_3_text, (5, 90),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # add text
    type_4_text = '{}: {}%' .format(class_names[3], int(type_4_pred*100))
    cv2.putText(frm, type_4_text, (5, 110),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    # add text
    type_5_text = '{}: {}%' .format(class_names[4], int(type_5_pred*100))
    cv2.putText(frm, type_5_text, (5, 140),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    # add text
    type_6_text = '{}: {}%' .format(class_names[5], int(type_6_pred*100))
    cv2.putText(frm, type_6_text, (5, 170),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    #add text
    type_7_text = '{}: {}%' .format(class_names[6], int(type_7_pred*100))
    cv2.putText(frm, type_7_text, (5, 200),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    type_8_text = '{}: {}%' .format(class_names[7], int(type_8_pred*100))
    cv2.putText(frm, type_8_text, (5, 230),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    type_9_text = '{}: {}%' .format(class_names[8], int(type_9_pred*100))
    cv2.putText(frm, type_9_text, (5, 260),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    type_10_text = '{}: {}%' .format(class_names[9], int(type_10_pred*100))
    cv2.putText(frm, type_10_text, (5, 290),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    type_11_text = '{}: {}%' .format(class_names[10], int(type_11_pred*100))
    cv2.putText(frm, type_11_text, (5, 310),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
    type_12_text = '{}: {}%' .format(class_names[11], int(type_12_pred*100))
    cv2.putText(frm, type_12_text, (5, 340),
              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
#     type_13_text = '{}: {}%' .format(class_names[12], int(type_13_pred*100))
#     cv2.putText(frm, type_13_text, (5, 370),
#               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
#     type_14_text = '{}: {}%' .format(class_names[13], int(type_14_pred*100))
#     cv2.putText(frm, type_14_text, (5, 400),
#               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
#     type_15_text = '{}: {}%' .format(class_names[14], int(type_15_pred*100))
#     cv2.putText(frm, type_15_text, (5, 430),
#               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
#     type_16_text = '{}: {}%' .format(class_names[15], int(type_16_pred*100))
#     cv2.putText(frm, type_16_text, (5, 450),
#               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
    
#     type_17_text = '{}: {}%' .format(class_names[16], int(type_17_pred*100))
#     cv2.putText(frm, type_17_text, (5, 480),
#               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)
   
    # show the frame
    cv2.imshow("TEST the frame( Photo )", frm)
    
    key = cv2.waitKey(1)
    
    # quit camera if 'q' is pressed
    if key & 0xFF == ord("q"):
        break
        

        
CAMERA.release()
cv2.destroyAllWindows()

In [ ]:
List_Id = class_names[np.argmax(predictions)]
print(List_Id)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from keras import preprocessing

In [ ]:
plt.figure(figsize=(15, 15))
x = 0
for i in range(36, 51):
    plt.subplot(5, 5, x+1)
    x = x + 1
    plt.xticks([])
    plt.yticks([])
    plt.grid('off')
    the_image = preprocessing.image.load_img('I:/Data sets/New Validation/117808_09/' + str(i) + '.png', target_size=(width, height))
    plt.imshow(the_image, cmap=plt.cm.binary)
    the_image = np.expand_dims(the_image, axis=0)
    predictions = model.predict(the_image)
    predicted_label = class_names[np.argmax(predictions)]
    true_label = '117808_09'
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    plt.xlabel("{} ({})".format(predicted_label, true_label), color=color)